## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [2]:
data = pd.read_csv('./data/AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
data.shape

(48895, 16)

In [4]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [5]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [6]:
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129
6,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,2017-10-05,0.40,1,0
7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220
8,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,2017-07-21,0.99,1,0
9,5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-06-09,1.33,4,188


Как видите, в данных есть пропуски. Не забудьте обработать их

Заполним пропуски в колонках **name** и **host_name** значением '-1', которое будет сигнализировать о пропуске в колонке.

In [7]:
data.loc[data['name'].isna(), 'name'] = '-1'
data.loc[data['host_name'].isna(), 'host_name'] = '-1'

С колонкой $last\_review$ поступим иначе:
* $last\_review$ - рассмотрим объект $x$ такой, что $x["last\_review"] = nan$ и $x["neighbourhood"] = neigh$. Тогда $x["last\_review"]$ заполняем наиболее встречаемым значением в колонке $last\_review$ среди всех объектов $x_{i}$ тренировочной выборки: $x_{i}["neighbourhood"] = neigh$

In [8]:
%%time
is_last_review_nan = data['last_review'].isna()
for neigh in data[is_last_review_nan]['neighbourhood']:
    is_neigh = data['neighbourhood'] == neigh
    if data[is_neigh].shape[0] == 1:
        data.loc[is_last_review_nan, 'last_review'] = '-1'
    else:
        data.loc[is_neigh & is_last_review_nan, 'last_review'] = data[is_neigh]['last_review'].value_counts().idxmax()

CPU times: user 1min 7s, sys: 12.7 ms, total: 1min 7s
Wall time: 1min 7s


А для колонки **$reviews\_per\_month$** возьмем не наиболее встречаемое значение, а среднее:

In [9]:
%%time
is_reviews_per_month_nan = data['reviews_per_month'].isna()
for neigh in data[is_last_review_nan]['neighbourhood']:
    is_neigh = data['neighbourhood'] == neigh
    if data[is_neigh].shape[0] == 1:
        data.loc[is_reviews_per_month_nan, 'reviews_per_month'] = -1
    else:
        data.loc[is_neigh & is_reviews_per_month_nan, 'reviews_per_month'] = data[is_neigh]['reviews_per_month'].mean()

CPU times: user 42.1 s, sys: 4.05 ms, total: 42.1 s
Wall time: 42.1 s


Разобъем данные на обучение и контроль.

Для тестирования модели без категориальных признаков:

In [10]:
from sklearn.model_selection import train_test_split
X_train_non_cat, X_test_non_cat, y_train_non_cat, y_test_non_cat = train_test_split(data.drop(columns=['price',
                                                                                              'last_review',
                                                                                              'name',
                                                                                              'host_name',
                                                                                              'neighbourhood_group',
                                                                                              'neighbourhood',
                                                                                              'room_type']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

Для тестирования только категориальных признаков:

In [11]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                        'id',
                                                                                                       'host_id',
                                                                                                       'latitude',
                                                                                                       'longitude',
                                                                                                       'minimum_nights',
                                                                                                       'number_of_reviews',
                                                                                                       'reviews_per_month',
                                                                                                       'calculated_host_listings_count',
                                                                                                       'availability_365']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']),
                                                    data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [13]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


class KNNRegressor:
    def __init__(self, n_neighbors: int,
                 metric: Union[str, Callable],
                 mode: str = 'uniform',
                test_block_size = 10):
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
            test_block_size: block size for finding neighbors and predict (for saving memory)
        """
        self.__nn = NearestNeighbors(n_neighbors=n_neighbors, metric=metric)
        self.__mode = mode
        self.__n_neighbors = n_neighbors
        self.test_block_size = test_block_size

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        self.y_train = y
        self.__nn.fit(X, y)
        self.eps = 1e-5

    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """
        if n_neighbors is not None:
            n_neighbors = np.minimum(self.__n_neighbors, n_neighbors)
        else:
            n_neighbors = self.__n_neighbors
        if self.test_block_size > X.shape[0]:
            self.test_block_size = X.shape[0]
        preds = np.zeros(X.shape[0])
        split_size = X.shape[0] // self.test_block_size + \
                     int(X.shape[0] % self.test_block_size != 0)
        curr_idx = 0
        for i, split in enumerate(np.array_split(X, split_size)):
            self.distances, self.neigh_idxs = self.__nn.kneighbors(split,
                                                                   n_neighbors,
                                                                   return_distance=True
                                                                  )
            for j, idx in enumerate(self.neigh_idxs):
                if self.__mode == 'distance':
                    weights = 1 / (self.distances[j] + self.eps)
                    pred = np.sum(self.y_train.values[idx] * weights) / np.sum(weights)
                else:
                    pred = np.sum(self.y_train.values[idx])
                preds[j + curr_idx] = pred / n_neighbors
            curr_idx += split.shape[0]
        return preds

In [14]:
def rmse(y_true, y_valid):
    return (np.sum(y_valid - y_true) ** 2 / len(y_true)) ** (1/2)

In [15]:
k_neighbors = 3
knn = KNNRegressor(k_neighbors, metric='euclidean', mode='uniform', test_block_size=500)

In [16]:
%%time
knn.fit(X_train_non_cat, y_train_non_cat)
preds = knn.predict(X_test_non_cat, 5)

CPU times: user 242 ms, sys: 0 ns, total: 242 ms
Wall time: 243 ms


In [17]:
rmse(y_test_non_cat.values.reshape(-1), preds)

230.62530710043163

In [18]:
#388.5047149768024 - euclidean, weighted
#163.30259925125785 - euclidean, weighted, 2
#230.62530710043154 - euclidean, weighted, 3

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [19]:
# import pyximport; pyximport.install() Попытки переписать метрики на Cython оказались безуспешными
# import overlap_cython

In [20]:
def overlap(x, z):
    return np.sum(x != z)

def flattened_overlap(x, z): # возможен баг
    f_x = np.zeros(x.shape[0])
    f_z = np.zeros(z.shape[0])
    p2_sum_arr = np.zeros(x.shape[0])
    for i, x_val in enumerate(x):
        f_x[i] = np.sum(x == x_val)
        i += 1
    p_x = f_x / x.shape[0]
    for i, x_val in enumerate(x):
        p2_sum_arr[i] = np.sum((f_x * (f_x - 1) / (x.shape[0] * (x.shape[0] - 1)))  * (p_x <= p_x[i])) ## деление или не
    result = np.sum(x != z) + np.sum((x == z)  * p2_sum_arr)
    return result

def log_overlap(x, z):
    result = 0
    f_x = np.zeros(x.shape[0])
    f_z = np.zeros(z.shape[0])
    i = 0
    for x_val, z_val in zip(x, z):
        f_x[i] = np.sum(x == x_val)
        f_z[i] = np.sum(z == z_val)
        i += 1
    result = np.sum((x != z) * np.log(f_x + 1) * np.log(f_z + 1))
    return result

In [21]:
overlap(np.array([1, 1, 1]), np.array([1, 1, 1]))

0

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
le = LabelEncoder()
for column in X_train_cat.columns:
    all_classes_in_col = np.hstack([X_train_cat[column].unique(), X_test_cat[column].unique()])
    le.fit(all_classes_in_col)
    X_train_cat.loc[:, column] = le.transform(X_train_cat.loc[:, column])
    X_test_cat.loc[:, column] = le.transform(X_test_cat.loc[:, column])


In [24]:
knn = KNNRegressor(10, metric=overlap, mode='uniform', test_block_size=10)
knn.fit(X_train_cat, y_train_cat)

In [25]:
%%time
preds = knn.predict(X_test_cat)

CPU times: user 33min 12s, sys: 14.1 s, total: 33min 26s
Wall time: 33min 17s


In [26]:
rmse(y_test_cat.values.reshape(-1), preds)

1849.6800797726262

In [27]:
#1849.6800797726262

In [28]:
knn = KNNRegressor(10, metric=flattened_overlap, mode='uniform', test_block_size=10)
knn.fit(X_train_cat, y_train_cat)

In [29]:
%%time
preds = knn.predict(X_test_cat)

KeyboardInterrupt: 

In [30]:
rmse(y_test_cat.values.reshape(-1), preds)

1849.6800797726262

In [31]:
knn = KNNRegressor(10, metric=log_overlaperlap, mode='uniform', test_block_size=10)
knn.fit(X_train_cat, y_train_cat)

In [ ]:
%%time
preds = knn.predict(X_test_cat)

In [ ]:
rmse(y_test_cat.values.reshape(-1), preds)

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [ ]:
# Ваш код здесь

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [ ]:
# Ваш код здесь

<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [ ]:
data['target'] = (data.price > data.price.mean()).astype(int)

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [ ]:
def counters(x):
    """
    params:
        x: value on categorical feature for N objects
    returns: vector of length N
    """
    
    counts = np.zeros(x.shape[0])
    for i, x_val in enumerate(x):
        counts[i] = np.sum(x == x_val)
    
    return counts
    
    

In [ ]:
counters([1, 2, 3, 4, 5])

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

In [ ]:
def fold_counters(x):
    """
    params:
        x: value on categorical feature for N objects
    returns: vector of length N
    """
    # Ваш код здесь
    pass

Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [ ]:
# Ваш код здесь

<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [ ]:
# Ваш код здесь

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

In [ ]:
# Ваш код здесь

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [ ]:
# Ваш код здесь

## Часть 4: Выводы


In [ ]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆